# PART 1

In [1]:
import numpy as np
import scipy.io

In [7]:
# mat = scipy.io.loadmat('/Users/yasinyokus/Desktop/CV HW3 -- 28ARALIK/BSR/BSDS500/data/groundTruth/test/107072.mat')
mat = scipy.io.loadmat('/Users/yasinyokus/Desktop/CV HW3 -- 28ARALIK/BSR/BSDS500/data/groundTruth/train/66039.mat')

In [8]:
# mat['groundTruth'][0, 1][0][0][0] # gray
# mat['groundTruth'][0, 1][0][0][1] # binary
# (mat['groundTruth'][0, 1][0][0][1] != 0).shape

In [9]:
import matplotlib.pyplot as plt
from functools import reduce
import cv2
arr_length = mat['groundTruth'].shape[1]
truths = [mat['groundTruth'][0, i][0][0][1] for i in range(arr_length)]
edge_ground_truth = reduce(np.add, truths) # use this as the edge ground truths

In [116]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from functools import reduce
import cv2

test_file_path = '/Users/yasinyokus/Desktop/CV HW3 -- 28ARALIK/BSR/BSDS500/data/groundTruth/test/107072.mat'
# test_file_path = scipy.io.loadmat('/Users/yasinyokus/Desktop/CV HW3 -- 28ARALIK/BSR/BSDS500/data/groundTruth/train/66039.mat')

def value_counts(image):
    unique, counts = np.unique(image, return_counts=True)
    return dict(zip(unique, counts))

def get_edge_ground_truth(file_path):
    mat = scipy.io.loadmat(file_path)
    new_mat = np.zeros_like(mat['groundTruth'][0, 0][0][0][1])
    print(new_mat.shape)
    arr_length = mat['groundTruth'].shape[1]
    truths = [mat['groundTruth'][0, i][0][0][1] for i in range(arr_length)]
    edge_ground_truth = reduce(np.logical_or, truths).astype(np.uint8) # use this as the edge ground truths
    return edge_ground_truth

# (321, 481)
# {0: 143546, 1: 10855}

# edge_ground_truth = get_edge_ground_truth(test_file_path)
# print(value_counts(edge_ground_truth))

# ret,edge_ground_truth = cv2.threshold(edge_ground_truth,0,1,cv2.THRESH_BINARY)
# print(edge_ground_truth.shape)
# plt.imshow(edge_ground_truth)

(321, 481)
{0: 143546, 1: 10855}


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [121]:
from os import listdir
from os.path import isfile, join
path_name_mat = '/Users/yasinyokus/Desktop/CV HW3 -- 28ARALIK/BSR/BSDS500/data/groundTruth/test/'
path_name_images = '/Users/yasinyokus/Desktop/CV HW3 -- 28ARALIK/BSR/BSDS500/data/images/test/'
# mat_files = [join(path_name_mat, f) for f in listdir(path_name_mat) if isfile(join(path_name_mat, f))]
image_files = [f for f in listdir(path_name_images) if isfile(join(path_name_images, f)) and f.endswith('.jpg')]

In [141]:
# fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, sharex=True, sharey=True)


def helper(image_file_name):
    image_file_path = join(path_name_images, image_file_name)
    file_name_wout_extension = image_file_name[:image_file_name.find('.jpg')]
    if file_name_wout_extension == -1:
        print("Something went wrong while finding file name!")
        return
    else:
        matrix_file_name = file_name_wout_extension + '.mat'
        mat_file_path = join(path_name_mat, matrix_file_name)
    # finding edge ground truths
    edge_ground_truth = get_edge_ground_truth(mat_file_path)
    # print('value_counts(edge_ground_truth): ', value_counts(edge_ground_truth))
    # ax1.imshow(edge_ground_truth)

    # Using Canny edge detection
    image_file = cv2.imread(image_file_path, cv2.IMREAD_GRAYSCALE)
    canny_edge_image = cv2.Canny(image_file, 100, 200)
    # ax2.imshow(canny_edge_image)
    return edge_ground_truth, canny_edge_image


def calculate_avg_precision(image_files):
    avg_precision = 0
    file_count = len(image_files)
    for image_file_name in image_files:
        edge_ground_truth, canny_edge_image = helper(image_file_name)
        rows, cols = edge_ground_truth.shape
        # – True positives: correctly detected object pixels
        # – True negatives: correctly detected non-object (background) pixels
        # – False positives: non-object pixels incorrectly detected as object pixels
        # – False negatives: object pixels missed by the object detector
        
        tpInd = (edge_ground_truth == 1) & (canny_edge_image == 255)
        unique, counts = np.unique(tpInd, return_counts=True)
        tp = dict(zip(unique, counts))[True]

        tnInd = (edge_ground_truth == 0) & (canny_edge_image == 0)
        unique, counts = np.unique(tnInd, return_counts=True)
        tn = dict(zip(unique, counts))[True]

        fpInd = (edge_ground_truth == 1) & (canny_edge_image == 0)
        unique, counts = np.unique(fpInd, return_counts=True)
        fp = dict(zip(unique, counts))[True]

        fnInd = (edge_ground_truth == 0) & (canny_edge_image == 255)
        unique, counts = np.unique(fnInd, return_counts=True)
        fn = dict(zip(unique, counts))[True]

        tpImage = np.zeros([rows, cols],dtype=np.uint8)
        tpImage[tpInd] = 255
        tnImage = np.zeros([rows, cols],dtype=np.uint8)
        tnImage[tnInd] = 255
        fpImage = np.zeros([rows, cols],dtype=np.uint8)
        fpImage[fpInd] = 255
        fnImage = np.zeros([rows, cols],dtype=np.uint8) 
        fnImage[fnInd] = 255

        # ax1.imshow(tpImage) 
        # ax2.imshow(tnImage) 
        # ax3.imshow(fpImage) 
        # ax4.imshow(fnImage)

        # Compute measures
        # accuracy = (tp + tn) / (tp + tn + fp + fn)
        # sens = tp/(tp + fn) 
        # spec = tn/(tn + fp)
        # print('Accuracy = ' + str(accuracy) + ', sensitivity = ' + str(sens) + ', specificity = ' + str(spec))
        prec = tp/(tp + fp)
        print('Precision = ' + str(prec))
        avg_precision += prec

    avg_precision /= file_count
    return avg_precision    


avg_precision = calculate_avg_precision(image_files)
print('avg_precision: ', avg_precision)


(481, 321)
Precision = 0.19991488456218748
(481, 321)
Precision = 0.28528844467823955
(481, 321)
Precision = 0.3055740828966174
(321, 481)
Precision = 0.2354724608388075
(481, 321)
Precision = 0.20348595933047448
(321, 481)
Precision = 0.2362663685723411
(481, 321)
Precision = 0.2862915760024868
(321, 481)
Precision = 0.1910974673829624
(321, 481)
Precision = 0.24011627906976743
(321, 481)
Precision = 0.2471730439748715
(481, 321)
Precision = 0.27203404859017555
(481, 321)
Precision = 0.18751808623516927
(481, 321)
Precision = 0.26583011583011584
(321, 481)
Precision = 0.2232620320855615
(321, 481)
Precision = 0.2543225400817353
(321, 481)
Precision = 0.2754754866417536
(481, 321)
Precision = 0.21064567769477055
(321, 481)
Precision = 0.0854380664652568
(321, 481)
Precision = 0.27327010888654724
(321, 481)
Precision = 0.11466563885141645
(321, 481)
Precision = 0.21991045562286016
(321, 481)
Precision = 0.31739707835325365
(481, 321)
Precision = 0.30490727892014635
(321, 481)
Precision 

In [ ]:
# Simple thresholding
for mat_file in mat_files:
    edge_ground_truth = get_edge_ground_truth(mat_file)
    ret,thresholded_image = cv2.threshold(edge_ground_truth,0,1,cv2.THRESH_BINARY)
    # print(edge_ground_truth.shape)
    # plt.imshow(edge_ground_truth)
    plt.imshow(thresholded_image)
    print(value_counts(thresholded_image))
    break
    # print(cv2.cvtColor(edge_ground_truth, cv2.COLOR_BGR2GRAY))


# Adaptive thresholding
for mat_file in mat_files:
    edge_ground_truth = get_edge_ground_truth(mat_file)
    print(value_counts(edge_ground_truth))
    # thresholded_image = cv2.adaptiveThreshold(edge_ground_truth,1,cv2.ADAPTIVE_THRESH_MEAN_C,\
    #         cv2.THRESH_BINARY,11,0)
    thresholded_image = cv2.adaptiveThreshold(edge_ground_truth,1,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,11,0)
    plt.imshow(thresholded_image)
    print(value_counts(thresholded_image))
    break


# # Otsu's thresholding
# for mat_file in mat_files:
#     edge_ground_truth = get_edge_ground_truth(mat_file)
#     print(value_counts(edge_ground_truth))
    
#     # Otsu's thresholding after Gaussian filtering
#     edge_ground_truth_blurred = cv2.GaussianBlur(edge_ground_truth,(5,5),0)
#     ret3,thresholded_image = cv2.threshold(edge_ground_truth_blurred,0,1,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
#     plt.imshow(thresholded_image)
#     print(value_counts(thresholded_image))
#     break